In [ ]:
from neo4j import GraphDatabase
import time

def get_driver():
  uri = "bolt+s://neo4j-partners.com:7687"
  return GraphDatabase.driver(uri, auth=("neo4j", "P@55w0rd@2023"))

def create_database(tx, dbname):
    tx.run("CREATE DATABASE $dbname", dbname=dbname)

driver = get_driver()
with driver.session(database="system") as session:
    try:
      session.execute_write(create_database, "neo4j")
      time.sleep(5)
    except Exception as e:
      print("Database already exists...")

with driver.session(database="neo4j") as session:
    session.run('CREATE CONSTRAINT IF NOT EXISTS FOR (c:Customer) REQUIRE (c.customerID) IS NODE KEY;')
    session.run('CREATE CONSTRAINT IF NOT EXISTS FOR (o:Order) REQUIRE (o.orderID) IS NODE KEY;')
    session.run('CREATE CONSTRAINT IF NOT EXISTS FOR (p:Product) REQUIRE (p.productID) IS NODE KEY;')
    session.run('CREATE CONSTRAINT IF NOT EXISTS FOR (s:Supplier) REQUIRE (s.supplierID) IS NODE KEY;')
    session.run('CREATE CONSTRAINT IF NOT EXISTS FOR (s:Employee) REQUIRE (s.employeeID) IS NODE KEY;')

driver.close()

print("database 'neo4j' created and constraints set")

In [ ]:
%%spark

var absfss_Base_Path="abfss://Neo4j_Workspace@onelake.dfs.fabric.microsoft.com/myLakehouse.Lakehouse/Files/Northwind/"
var url = "bolt+s://neo4j-partners.com:7687"
var password = "P@55w0rd@2023"

import org.apache.spark.sql.{SaveMode}
val df_customer = spark.read.option("header",true).csv(absfss_Base_Path+"customers.csv")
val df_supplier = spark.read.option("header",true).csv(absfss_Base_Path+"suppliers.csv")
val df_order = spark.read.option("header",true).csv(absfss_Base_Path+"orders.csv")
val df_order_detail = spark.read.option("header",true).csv(absfss_Base_Path+"order-details.csv")
val df_product = spark.read.option("header",true).csv(absfss_Base_Path+"products.csv")
val df_category = spark.read.option("header",true).csv(absfss_Base_Path+"categories.csv")

(df_customer.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("labels", ":Customer")
    .option("node.keys","customerID")
    .save())
(df_supplier.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("labels", ":Supplier")
    .option("node.keys","supplierID")
    .save())
(df_order.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("labels", ":Order")
    .option("node.keys","orderID")
    .save())
(df_order_detail.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("labels", ":Order_Detail")
    .option("node.keys","orderID")
    .save())
(df_product.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("labels", ":Product")
    .option("node.keys","productID")
    .save())
(df_category.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("labels", ":Category")
    .option("node.keys","categoryID")
    .save())
(df_product.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("relationship.save.strategy", "keys")
    .option("relationship", "PRODUCT-CATERGORY")
    .option("relationship.source.labels", ":Product")
    .option("relationship.source.node.keys", "productID:productID")
    .option("relationship.target.labels", ":Category")
    .option("relationship.target.node.keys", "categoryID:categoryID")
    .save())
(df_product.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("relationship.save.strategy", "keys")
    .option("relationship", "SUPPLIER-PRODUCT")
    .option("relationship.source.labels", ":Supplier")
    .option("relationship.source.node.keys", "supplierID:supplierID")
    .option("relationship.target.labels", ":Product")
    .option("relationship.target.node.keys", "productID:productID")
    .save())
(df_order_detail.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("relationship.save.strategy", "keys")
    .option("relationship", "ORDER-PRODUCT")
    .option("relationship.source.labels", ":Order")
    .option("relationship.source.node.keys", "orderID:orderID")
    .option("relationship.target.labels", ":Product")
    .option("relationship.target.node.keys", "productID:productID")
    .save())
(df_order.write.format("org.neo4j.spark.DataSource")
    .mode(SaveMode.Overwrite).option("url",url)
    .option("authentication.basic.username", "neo4j")
    .option("authentication.basic.password",password)
    .option("relationship.save.strategy", "keys")
    .option("relationship", "CUSTOMER-ORDERED")
    .option("relationship.source.labels", ":Customer")
    .option("relationship.source.node.keys", "customerID:customerID")
    .option("relationship.target.labels", ":Order")
    .option("relationship.target.node.keys", "orderID:orderID")
    .save())